# SOM : carte autoadaptive : DÃ©tection de Fraude

## Partie 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Importation des données
dataset = pd.read_csv('Credit_Card_Applications.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [ ]:
# Changement d'échelle
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
X = sc.fit_transform(X)

In [ ]:
# Entrainement SOM
from minisom import MiniSom
som = MiniSom(x = 10, y = 10, input_len = 15, sigma = 1.0, learning_rate = 0.5)  # tableau de 10x10 avec longeur de X = 15

som.random_weights_init(X)   # Initisaliser aléatoirement les poids
som.train_random(X, num_iteration = 100)

### Visualisation des résultats

In [ ]:
# calcul du distance BMI
    
from pylab import bone, pcolor, colorbar, plot, show

bone()  # Initialise un graph

pcolor(som.distance_map().T)
colorbar() # légende

# Fraude = Blanc

markers = ["o", "s"]
colors = ["r", "g"]
for i, x in enumerate(X):
    w = som.winner(x)
    plot(w[0] + 0.5,
         w[1] + 0.5,
         markers[y[i]],
         markeredgecolor = colors[y[i]],
         markerfacecolor = 'None',
         markersize = 10,
         markeredgewidth = 2)

show()

In [ ]:
# Détecter la fraude 
mappings = som.win_map(X)
frauds = np.concatenate((mappings[(5,8)], mappings[(3,7)]), axis = 0)  #axis = 0 par les lignes
frauds = sc.inverse_transform(frauds)

## Partie 2

In [ ]:
# Créer la matrice de variables
customers = dataset.iloc[:, 1: ].values

In [ ]:
# Créer une variable dépendante
is_fraud = np.zeros(len(dataset))

In [ ]:
for i in range(len(dataset)) :
    if dataset.iloc[i,0] in frauds :
        is_fraud[i] = 1

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [ ]:
customers = sc.fit_transform(customers)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

classifier = Sequential()

# Ajouter la couche d'entrée
classifier.add(Dense(units=2, activation="relu",kernel_initializer="uniform", input_dim=15))

# Ajouter la couche de sortie
classifier.add(Dense(units=1, activation="sigmoid", kernel_initializer="uniform"))   #activation= softmax si plusieurs units en sortie


In [ ]:
# Complier les neurones
classifier.compile(loss= tf.keras.losses.BinaryCrossentropy(), 
                   optimizer= tf.keras.optimizers.Adam(), 
                   metrics= [tf.keras.metrics.Accuracy()])


In [ ]:
# Entrainer le réseau de neuronnes
classifier.fit(customers, is_fraud, batch_size=1, epochs=10) 

In [ ]:
# Prédictions

y_pred = classifier.predict(customers)

y_pred = np.concatenate( (dataset.iloc[:, 0:1], y_pred), axis=1)  # 0:1 permet de créer un tableau a 2D avec seulement la premiere colonne pour réaliser la concat


y_pred = y_pred[y_pred[:,1].argsort()]  # ordonner 